In [1]:
import sys
import os
import scipy.io as sio
import cv2
import torch
import time
import numpy as np
import onnx
import onnxruntime
import MNN
import torch.nn.functional as F
import torch.nn as nn
from model.dbface_small import DBFace as dbface
from model.dbface_light import DBFace as dbface_light
from utils import preprocess
from model.losses import GIoULoss
from tqdm import tqdm
import pandas as pd
import random

#### 构建模型

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
model = dbface(has_landmark=True, wide=64, has_ext=True, upmode="UCBA")
model.eval()
model.cuda()

DBFace(
  (bb): Mbv3SmallFast(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (hs1): ReLU(inplace=True)
    (bneck): Sequential(
      (0): Block(
        (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (nolinear1): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (nolinear2): ReLU(inplace=True)
        (conv3): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): Block(
        (conv1): Conv2d(16,

In [6]:
from torchsummary import summary
summary(model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             432
       BatchNorm2d-2         [-1, 16, 128, 128]              32
              ReLU-3         [-1, 16, 128, 128]               0
            Conv2d-4         [-1, 16, 128, 128]             256
       BatchNorm2d-5         [-1, 16, 128, 128]              32
              ReLU-6         [-1, 16, 128, 128]               0
            Conv2d-7           [-1, 16, 64, 64]             144
       BatchNorm2d-8           [-1, 16, 64, 64]              32
              ReLU-9           [-1, 16, 64, 64]               0
           Conv2d-10           [-1, 16, 64, 64]             256
      BatchNorm2d-11           [-1, 16, 64, 64]              32
            Block-12           [-1, 16, 64, 64]               0
           Conv2d-13           [-1, 72, 64, 64]           1,152
      BatchNorm2d-14           [-1, 72,

In [8]:
for name, value in model.named_parameters():
    print (name)

bb.conv1.weight
bb.bn1.weight
bb.bn1.bias
bb.bneck.0.conv1.weight
bb.bneck.0.bn1.weight
bb.bneck.0.bn1.bias
bb.bneck.0.conv2.weight
bb.bneck.0.bn2.weight
bb.bneck.0.bn2.bias
bb.bneck.0.conv3.weight
bb.bneck.0.bn3.weight
bb.bneck.0.bn3.bias
bb.bneck.1.conv1.weight
bb.bneck.1.bn1.weight
bb.bneck.1.bn1.bias
bb.bneck.1.conv2.weight
bb.bneck.1.bn2.weight
bb.bneck.1.bn2.bias
bb.bneck.1.conv3.weight
bb.bneck.1.bn3.weight
bb.bneck.1.bn3.bias
bb.bneck.2.conv1.weight
bb.bneck.2.bn1.weight
bb.bneck.2.bn1.bias
bb.bneck.2.conv2.weight
bb.bneck.2.bn2.weight
bb.bneck.2.bn2.bias
bb.bneck.2.conv3.weight
bb.bneck.2.bn3.weight
bb.bneck.2.bn3.bias
bb.bneck.3.se.se.0.weight
bb.bneck.3.se.se.1.weight
bb.bneck.3.se.se.1.bias
bb.bneck.3.se.se.3.weight
bb.bneck.3.se.se.4.weight
bb.bneck.3.se.se.4.bias
bb.bneck.3.conv1.weight
bb.bneck.3.bn1.weight
bb.bneck.3.bn1.bias
bb.bneck.3.conv2.weight
bb.bneck.3.bn2.weight
bb.bneck.3.bn2.bias
bb.bneck.3.conv3.weight
bb.bneck.3.bn3.weight
bb.bneck.3.bn3.bias
bb.bneck.4.se.

#### 读取标注文件

In [4]:
def load_retina_file(file_path):
    with open(file_path, 'r') as f:
        data = f.readlines()
        
    name_list = []
    lc_list = []
    w_list, h_list = [], []
    landmark_list = []
    for line in data:
        line = line.strip('\n')
        if line.startswith('#'):
            name = line[2:]
        else:
            line = line.split(' ')
            name_list.append(name)
            lc_list.append([int(float(line[i])) for i in range(2)])
            w_list.append(int(float(line[2])))
            h_list.append(int(float(line[3])))
            landmark = [[0,0] for i in range(5)]
            for i in range(5):
                x, y = line[i * 3 + 4:i * 3 + 6]
                landmark[i][0], landmark[i][1] = float(x), float(y)
            landmark_list.append(landmark)
    retina_df = pd.DataFrame(data={'name':name_list,
                                   'left_coor':lc_list,
                                   'w':w_list,
                                   'h':h_list,
                                   'landmarks':landmark_list})
    return retina_df

In [8]:
df = load_retina_file('/home/data/Datasets/WIDERFace/clean_widerface_50_.txt')
df_20_50 = df[(df['w']<50)&(df['w']>20)&(df['h']<50)&(df['h']>20)]

In [9]:
df_20_50

,name,left_coor,w,h,landmarks
14,WIDERFace/WIDER_train/images/0--Parade/0_Parad...,"[642, 320]",23,22,"[[653.24, 323.06], [661.89, 323.67], [661.44, ..."
16,WIDERFace/WIDER_train/images/0--Parade/0_Parad...,"[794, 344]",21,22,"[[804.46, 348.43], [812.71, 348.24], [811.75, ..."
25,WIDERFace/WIDER_train/images/0--Parade/0_Parad...,"[296, 178]",45,41,"[[306.88, 185.12], [326.0, 184.75], [315.12, 1..."
64,WIDERFace/WIDER_train/images/0--Parade/0_Parad...,"[22, 248]",25,30,"[[34.27, 254.95], [44.48, 254.36], [43.89, 260..."
67,WIDERFace/WIDER_train/images/0--Parade/0_Parad...,"[408, 210]",25,30,"[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-1..."
...,...,...,...,...,...
111002,WIDERFace/WIDER_train/images/22--Picnic/22_Pic...,"[361, 94]",48,47,"[[375.78, 104.1], [399.08, 102.01], [391.04, 1..."
111781,WIDERFace/WIDER_train/images/37--Soccer/37_Soc...,"[655, 125]",49,47,"[[665.93, 138.15], [682.84, 133.51], [671.09, ..."
112047,WIDERFace/WIDER_train/images/40--Gymnastics/40...,"[414, 160]",49,47,"[[429.07, 169.69], [450.49, 169.1], [444.17, 1..."
112463,WIDERFace/WIDER_train/images/47--Matador_Bullf...,"[559, 85]",46,48,"[[564.76, 98.86], [580.07, 97.47], [567.45, 11..."


#### 观察样本

In [10]:
def draw_face(root, df, num):
    name = df.iloc[num]['name']
    lc = df.iloc[num]['left_coor']
    w = df.iloc[num]['w']
    h = df.iloc[num]['h']
    landmark = df.iloc[num]['landmarks']
    img = cv2.imread(os.path.join(root, name))
    try:
        for j in range(len(landmark)):
            cv2.circle(img, (int(landmark[j][0]), int(landmark[j][1])), 2, (255, 0, 0), -1)
    except:
        pass
    cv2.rectangle(img, (lc[0], lc[1]), (lc[0]+w, lc[1]+h), (0, 255, 0), 1)
    cv2.imshow('im',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return

In [17]:
root = '/home/data/Datasets/'
for i in range(10):
    num = random.randint(0, 27379)
    draw_face(root, df_20_50, num)

#### 配置输入

In [9]:
mean = [0.408, 0.447, 0.47]
std = [0.289, 0.274, 0.278]
image = cv2.imread("/home/data/TestImg/tuchong/5.jpg")
image = ((image/255.-mean)/std).astype(np.float32)
image = cv2.resize(image,(256, 256))
input = image.transpose(2, 0, 1)
input = torch.from_numpy(input)[None]

In [7]:
net_time = time.time()
for i in range(1000):
    model(input)
print (time.time()-net_time)

17.68730878829956


In [12]:
torch.save(model, './model/model_file/test_model.pth')

### 在私有test数据集上进行测试

In [ ]:
def trans2dbform(file,output_file):
    # 21 38 57 58 62
    with open(file, 'r') as f:
        data = f.readlines()
    f = open(output_file, 'a')
    for line in data:
        if line[0]=='#':
            f.write(line)
        else:
            line = line.split(' ')
            line = line[0]+' '+line[1]+' '+str(int(line[2])-int(line[0]))+' '+str(int(line[3])-int(line[1]))+' '+line[49]+' '+line[50]+' 0 '+line[83]+' '+line[84]+' 0 '+line[121]+' '+line[122]+' 0 '+line[123]+' '+line[124]+' 0 '+line[131]+' '+line[132]+' 0 1\n'
            f.write(line)

In [ ]:
model = dbface(has_landmark=True, wide=64, has_ext=True, upmode="UCBA")
model.load('./model/model_file/dbface_nearsmall_rubust_selfdata5.pth')
model.eval()
model.cuda()
mean = [0.408, 0.447, 0.47]
std = [0.289, 0.274, 0.278]
label_file = '/home/data/Datasets/SD/self_test/label.txt'
images_dir = '/home/data/Datasets/SD/self_test/images/'
giou_loss = GIoULoss()

In [ ]:
def eval_reg(gt):
    imgfile, objs = gt
    img = cv2.imread(imgfile)
    H, W, _ = img.shape
    mhw = max(H, W)
    scale = mhw / 256
    new_img = np.zeros((mhw, mhw, 3), np.float32)
    reg_tlrb = np.zeros((1 * 4, 64, 64), np.float32)
    reg_mask = np.zeros((1, 64, 64), np.float32)
    new_img[:H, :W, :] = img
    img = cv2.resize(new_img, (256,256))
    
    for obj in objs:
        obj.x = obj.x / scale
        obj.y = obj.y / scale
        obj.r = obj.r / scale
        obj.b = obj.b / scale
        cx, cy = obj.safe_scale_center(1 / 4, 64, 64)
        reg_box = np.array(obj.box) / 4
        reg_tlrb[:, cy, cx] = reg_box
        reg_mask[0, cy, cx] = 1
        
    reg_tlrb = torch.tensor(reg_tlrb).unsqueeze(0).cuda()
    reg_mask = torch.tensor(reg_mask).unsqueeze(0).cuda()
    img = ((img/255.-mean)/std).astype(np.float32)
    img = cv2.resize(img,(256, 256))
    img = img.transpose(2, 0, 1)
    img = torch.from_numpy(img)[None].cuda()
    hm, pred_tlrb, _  = model(img)
    reg_loss = giou_loss(pred_tlrb, reg_tlrb, reg_mask)
    return reg_loss
#     return reg_tlrb, pred_tlrb

In [ ]:
test_gt = preprocess.load_webface(label_file, images_dir)
reg_loss = []
for i in tqdm(range(len(test_gt))):
    reg_loss.append(eval_reg(test_gt[i]).data.cpu().numpy())
reg_loss

In [ ]:
1-np.array(reg_loss).mean()

In [20]:
import torch
a = torch.Tensor([1]).cuda()
b = torch.Tensor([2]).cuda()

In [31]:
c = []
c.append(a)
c.append(b)

In [32]:
c = torch.Tensor(c)

In [41]:
print (f"{1-c.mean()}")

-0.5
